In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='06-sklearn.tree'
classifierName='01-DecisionTreeClassifier'
extraParameterName='3-Random undersampling'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
# Hyperparameters Results
# not too high to avoid overfitting (lower or equals to 12 ?)
max_depthFound=9
min_samples_leafFound=22
min_samples_splitFound=19
criterionFound='entropy'

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound)


print('done')

In [ ]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)
dfTrx.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx0)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train, y_train = sp.randomUnderSampling(x_train0, y_train0)


In [ ]:
%%script false
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)
dic_param={
    'criterion':["gini","entropy"],
    'max_depth': randint(8,17),
    'min_samples_leaf': randint(10,25),
    'min_samples_split': randint(15,25),
}

res=fd.hyperparameterSelectionRandomizedSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train,iter=20)
print(res)


#{'criterion': 'entropy', 'max_depth': 11, 'min_samples_leaf': 19, 'min_samples_split': 19}
#0.4512754432694376
#score   0.5173463177115033

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 22, 'min_samples_split': 21}
#0.45170459069654845
#score   0.5106124924196482


In [ ]:
#%%script false
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

modelClf = DecisionTreeClassifier(random_state=42)

dic_param={
    'criterion':["gini","entropy"],
    'min_samples_leaf': [21,22,23],
    'min_samples_split': [19,20,21],
    'max_depth': [8,9,10]
}



res=fd.hyperparameterSelectionGridSearchCVSampling(modelClf, dic_param, 'f1', x_train, y_train)
print(res)



#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 22, 'min_samples_split': 20}
#0.45170459069654845
#scoref1 0.5106124924196482

#{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 22, 'min_samples_split': 19}
#0.45170459069654845
#scoref1 0.5106124924196482



In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx0)


modelClf = DecisionTreeClassifier(random_state=42)
parameters={'criterion': criterionFound, 'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 'min_samples_split': min_samples_splitFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
#resultMd.update_time_response_result('06-sklearn.tree','DecisionTree','2-After tuning', duration_in_s)
f1Test,mccTest,rocTest=fd.print_scores(y_test, predsTest,'all')
#resultMd.update_performance_test_result('06-sklearn.tree','DecisionTree','2-After tuning', f1,mcc,roc)

f1Train=fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD06A-DecisionTreeClassifierProbaHistogram' if saveImage else None)

diffF1=f1Train-f1Test
print("diffF1",diffF1)

tmp = pd.DataFrame({'Feature': predictors, 'Feature importance': modelClf.feature_importances_})
val=tmp[(tmp['Feature']=='trx_amount_log10')]['Feature importance']
amountImportance =val.values[0]
print("amount importance",amountImportance)

In [ ]:
files = fd.getAllFiles()
predictors = fd.getPredictors(dfTrx0)

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx=fd.getStandardScaledData(file,source,predictors)
    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3],diffF1,amountImportance)

